# Preprocessing of MAGeT results and merge into the matched data table.
## Input:
    matched_des_data_file  = data_dir/'matched_Des-cere_qced.csv'
    matched_DKT_data_file  = data_dir/'matched_DKT-cere_qced.csv'
## Ouput:
    matched_des_all_file = data_dir/ 'matched_Des-all_qced.csv'
    matched_DKT_all_file = data_dir/ 'matched_DKT-all_qced.csv'

In [1]:
#reading/merging/selecting data
from pathlib import Path
import nipype.interfaces.io as nio
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# input folder
data_dir = Path("/codes/tab_data")
maget_dir = Path("/codes/preproc/maget/results")

# Existing data file
matched_des_data_file  = data_dir/'matched_Des-cere_qced.csv'
matched_DKT_data_file  = data_dir/'matched_DKT-cere_qced.csv'

# output file
matched_des_all_file = data_dir/ 'matched_Des-all_qced.csv'
matched_DKT_all_file = data_dir/ 'matched_DKT-all_qced.csv'

cohorts_str = ['ADNI','ET','NC','PPMI']
# preproce of MAGeT resutls
et_maget_read = pd.read_csv(maget_dir/('ET_volumes.csv'), sep=',', header=0, index_col=0)
et_maget_read.index= [x[2:10] for x in et_maget_read.index]
nc_maget_read = pd.read_csv(maget_dir/('NC_volumes.csv'), sep=',', header=0, index_col=0)
nc_maget_read.index= [x[2:10] for x in nc_maget_read.index]
adni_maget_read = pd.read_csv(maget_dir/('ADNI_volumes.csv'), sep=',', header=0, index_col=0)
ppmi_maget_read = pd.read_csv(maget_dir/('PPMI_volumes.csv'), sep=',', header=0, index_col=0)
# remove sub-3600 from PPMI for not being selected and also conflicting with ET subject ID
ppmi_maget_read=ppmi_maget_read.drop(index=['sub-3600'])
maget_df = pd.concat([et_maget_read,nc_maget_read,ppmi_maget_read,adni_maget_read])
maget_df.index= [x.replace('-','_') for x in maget_df.index]


maget_df.loc[:,'L_I_IV']  = maget_df.loc[:,'L_I_II']+maget_df.loc[:,'L_III']+maget_df.loc[:,'L_IV']
maget_df.loc[:,'R_I_IV'] = maget_df.loc[:,'R_I_II']+maget_df.loc[:,'R_III']+maget_df.loc[:,'R_IV']

des_df = pd.read_csv(matched_des_data_file , sep=',', header=0, index_col=0);
DKT_df = pd.read_csv(matched_DKT_data_file , sep=',', header=0, index_col=0);

des_all_df = des_df.join(maget_df, how='left').copy()
DKT_all_df = DKT_df.join(maget_df, how='left').copy()

220115-00:14:03,909 nipype.utils WARNING:
	 A newer version (1.7.0) of nipy/nipype is available. You are using 1.5.0


# Adding Gray Matter volume from SUIT and MAGET

In [2]:
# adding gray matter volume from suit and maget segmentations
SUIT_l_labels = ['Left_I_IV',  'Left_V', 'Left_VI', 'Left_CrusI', 'Left_CrusII', 'Left_VIIb', 'Left_VIIIa', 'Left_VIIIb', 'Left_IX', 'Left_X'];
SUIT_r_labels = ['Right_I_IV','Right_V','Right_VI','Right_CrusI','Right_CrusII','Right_VIIb','Right_VIIIa','Right_VIIIb','Right_IX','Right_X'];
MAGeT_l_labels = ['L_I_IV',  'L_V', 'L_VI', 'L_Crus_I', 'L_Crus_II', 'L_VIIB', 'L_VIIIA', 'L_VIIIB', 'L_IX', 'L_X'];  
MAGeT_r_labels = ['R_I_IV','R_V','R_VI','R_Crus_I','R_Crus_II', 'R_VIIB','R_VIIIA','R_VIIIB','R_IX','R_X'];
# suit
des_all_df.loc[:,'Left_cerebellar_GM_SUIT'] =des_all_df.loc[:, SUIT_l_labels].sum(axis=1); des_all_df.loc[:,'Right_cerebellar_GM_SUIT'] =des_all_df.loc[:,SUIT_r_labels].sum(axis=1)
des_all_df.loc[:,'Left_cerebellar_GM_MAGeT']=des_all_df.loc[:,MAGeT_l_labels].sum(axis=1); des_all_df.loc[:,'Right_cerebellar_GM_MAGeT']=des_all_df.loc[:,MAGeT_r_labels].sum(axis=1)
# maget
DKT_all_df.loc[:,'Left_cerebellar_GM_SUIT'] =DKT_all_df.loc[:, SUIT_l_labels].sum(axis=1); DKT_all_df.loc[:,'Right_cerebellar_GM_SUIT'] =DKT_all_df.loc[:,SUIT_r_labels].sum(axis=1)
DKT_all_df.loc[:,'Left_cerebellar_GM_MAGeT']=DKT_all_df.loc[:,MAGeT_l_labels].sum(axis=1); DKT_all_df.loc[:,'Right_cerebellar_GM_MAGeT']=DKT_all_df.loc[:,MAGeT_r_labels].sum(axis=1)

# Save files

In [3]:
#save data
des_all_df.to_csv(matched_des_all_file)
DKT_all_df.to_csv(matched_DKT_all_file)